In [24]:
import sys
sys.path += ["../src"]
import simulator_BC as sim_bc
import estimation_epsilon_BC as est_bc
from calibrator_blackit import Calibrator #i just commented few lines of code to remove the output print
import numpy as np
import torch
from scipy.special import expit
import matplotlib.pyplot as plt

In [5]:
#define a simulator class for simulating the BC model knowing the real edges and opinions
#the calibration will compare the arrays of signs: self.s and simulate_s([epsilon])
#i input theta = [epsilon], instead of epsilon, because the Calibrator of black-it is designed for calibrating a list of parameters
class BC_simulator_given_previous_time():
    def __init__(self, X, edges, mu, real_epsilon, rho = 100, seed = 1):
        self.X = X
        self.edges = edges
        self.mu = mu
        self.real_epsilon = real_epsilon
        self.rho = rho
        self.seed = seed
        self.s = np.concatenate(np.array(edges[:,:,2]))[:, None] #Calibrator requires a time series of dim N, n_of_parameters
    
    def simulate_s(self, theta, T = 200, seed = None):
        epsilon, = theta
        
        s_pred = np.array([])
        
        if seed != None:
            np.random.seed(self.seed)
        
        T, edge_per_t, _ = self.edges.size()
        
        for t in range(T):
            for new_edge in range(edge_per_t):
                
                u,v,s = self.edges[t, new_edge]
                dist = np.abs(self.X[t, u] - self.X[t, v])
                
                if np.random.random() < expit(self.rho * (epsilon - dist)):
                    s_pred = np.append(s_pred, 1)
                else:
                    s_pred = np.append(s_pred, 0)
        
        
        return s_pred[:, None] #Calibrator requires a time series of dim N, n_of_parameters

In [6]:
N, T, edge_per_t = 100, 400, 3

epsilon, mu = 0.3, 0.6
epsilon_sim = 0.4

epsilon_bounds = [[0.], [1.]]
precisions = [0.0001]

In [7]:
X, edges = sim_bc.simulator_stoch(N, T, edge_per_t, epsilon, mu, seed = 1004, steepness = 100)
s_edges = np.concatenate(np.array(edges[:,:,2]))[:,None]

In [8]:
s_edges

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [10]:
from black_it.loss_functions.msm import MethodOfMomentsLoss
from numpy.typing import NDArray
from scipy.stats import kurtosis, skew
import statsmodels.api as sm

In [11]:
def get_mom_ts_1d(time_series: NDArray[np.float64]) -> NDArray[np.float64]:
    """Compute specific moments from a time series.

    Args:
        time_series: the time series

    Returns:
        the moments from a time series
    """
    # array of all moments (to be filled)
    avg_vec_mom = np.zeros(18)

    # first 4 moments and auto-correlations of the time series
    avg_vec_mom[0] = np.mean(time_series)
    avg_vec_mom[1] = np.std(time_series)
    s, k = skew(time_series), kurtosis(time_series)  # pylint: disable=invalid-name
    avg_vec_mom[2] = np.sign(s) * np.power(abs(s), 1.0 / 3.0)
    avg_vec_mom[3] = np.sign(k) * np.power(abs(k), 1.0 / 4.0)

    ts_acf = sm.tsa.acf(time_series, nlags=5, fft=False)
    avg_vec_mom[4] = ts_acf[1]
    avg_vec_mom[5] = ts_acf[2]
    avg_vec_mom[6] = ts_acf[3]
    avg_vec_mom[7] = ts_acf[4]
    avg_vec_mom[8] = ts_acf[5]

    # first 4 moments and auto-correlations of the absolute differences of the time series
    abs_diff = np.absolute(np.diff(time_series))

    avg_vec_mom[9] = np.mean(abs_diff)
    avg_vec_mom[10] = np.std(abs_diff)
    s, k = skew(abs_diff), kurtosis(abs_diff)  # pylint: disable=invalid-name
    avg_vec_mom[11] = np.sign(s) * np.power(abs(s), 1.0 / 3.0)
    avg_vec_mom[12] = np.sign(k) * np.power(abs(k), 1.0 / 4.0)

    ts_diff_acf = sm.tsa.acf(abs_diff, nlags=5, fft=False)
    avg_vec_mom[13] = ts_diff_acf[1]
    avg_vec_mom[14] = ts_diff_acf[2]
    avg_vec_mom[15] = ts_diff_acf[3]
    avg_vec_mom[16] = ts_diff_acf[4]
    avg_vec_mom[17] = ts_diff_acf[5]

    np.nan_to_num(avg_vec_mom, False)

    return avg_vec_mom


In [12]:
time_series = s_edges[:,0]

### first 4 moments 

In [13]:
np.mean(time_series)

0.7677527151211362

In [14]:
np.std(time_series)

0.4222658919984654

In [15]:
s,k = skew(time_series), kurtosis(time_series)  # pylint: disable=invalid-name

In [16]:
np.sign(s) * np.power(abs(s), 1.0 / 3.0)

-1.0824120671221904

In [17]:
np.sign(k) * np.power(abs(k), 1.0 / 4.0)

-0.7911339136973325

### auto-correlations of the time series

In [32]:
ts_acf = sm.tsa.acf(time_series, nlags = 5, fft = False)

In [33]:
ts_acf[1:6]

array([0.17622938, 0.16660614, 0.18977973, 0.15204492, 0.20441822])

### first 4 moments of the absolute differences of the time series 

In [34]:
abs_diff = np.absolute(np.diff(time_series))

In [35]:
s, k = skew(abs_diff), kurtosis(abs_diff)  # pylint: disable=invalid-name
np.mean(abs_diff), np.std(abs_diff), np.sign(s) * np.power(abs(s), 1.0 / 3.0), np.sign(k) * np.power(abs(k), 1.0 / 4.0)

(0.29347826086956524,
 0.45535565360116115,
 0.9680140899533785,
 -1.041629567248848)

### auto-correlations of the absolute differences of the time series

In [36]:
ts_diff_acf = sm.tsa.acf(abs_diff, nlags=5, fft=False)
ts_diff_acf[1:6]

array([0.28306273, 0.24239107, 0.22591402, 0.19852309, 0.24253256])

In [37]:
get_mom_ts_1d(time_series)

array([ 0.76775272,  0.42226589, -1.08241207, -0.79113391,  0.17622938,
        0.16660614,  0.18977973,  0.15204492,  0.20441822,  0.29347826,
        0.45535565,  0.96801409, -1.04162957,  0.28306273,  0.24239107,
        0.22591402,  0.19852309,  0.24253256])

## Loss MSM 

In [38]:
epsilon

0.3

In [43]:
simulator_BC = BC_simulator_given_previous_time(X, edges, mu, epsilon)
s_preds = [simulator_BC.simulate_s([x]) for x in np.arange(0, 1, 0.05)]

In [1]:
real_mom_1d = get_mom_ts_1d(s_edges[:,0])
sim_mom_1ds = [get_mom_ts_1d(s_pred[:,0]) for s_pred in s_preds]

In [45]:
gs = [real_mom_1d - sim_mom_1d for sim_mom_1d in sim_mom_1ds]

In [47]:
[(x, g.dot(g)) for x, g in zip(np.arange(0, 1, 0.05), gs)]

[(0.0, 22.305713510660194),
 (0.05, 4.854157371426787),
 (0.1, 3.3953409763627875),
 (0.15000000000000002, 0.5499549414692648),
 (0.2, 0.2444325628626215),
 (0.25, 0.09441310630708981),
 (0.30000000000000004, 0.002587663623467827),
 (0.35000000000000003, 2.9207078542590095),
 (0.4, 4.731813945264391),
 (0.45, 9.759280019964057),
 (0.5, 13.60139672563014),
 (0.55, 18.07127973545429),
 (0.6000000000000001, 21.752293952631536),
 (0.65, 24.567921551928624),
 (0.7000000000000001, 34.14316098095408),
 (0.75, 49.37409394101152),
 (0.8, 55.90911111937709),
 (0.8500000000000001, 89.5121129705198),
 (0.9, 4.7931614898544765),
 (0.9500000000000001, 4.7931614898544765)]